In [9]:
import PIL
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image,ImageOps,ImageFilter
import keras
from keras import layers
import os
from tensorflow.keras.utils import Sequence
from tensorflow.keras import Sequential
import tensorflow as tf
from keras.preprocessing import image  
import cv2
import pickle

In [10]:
from keras.models import load_model
model = load_model('model/vgg16/distracted-132-0.92.hdf5')

In [11]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(64, 64))
    # convert PIL.Image.Image type to 3D tensor with shape (64, 64, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 64, 64, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [12]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True 

test_tensor = path_to_tensor("car_test_5.jpg").astype('float32')/255 - 0.5

In [13]:
y_pred = model.predict_classes(test_tensor)

ValueError: in user code:

    C:\Users\SRIRAM\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1544 predict_function  *
        return step_function(self, iterator)
    C:\Users\SRIRAM\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1527 run_step  *
        outputs = model.predict_step(data)
    C:\Users\SRIRAM\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1500 predict_step  *
        return self(x, training=False)
    C:\Users\SRIRAM\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\base_layer.py:989 __call__  *
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    C:\Users\SRIRAM\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\sequential.py:375 call  *
        return super(Sequential, self).call(inputs, training=training, mask=mask)
    C:\Users\SRIRAM\AppData\Local\Programs\Python\Python39\lib\site-packages\

In [76]:
ypred_class = np.argmax(y_pred)
print(ypred_class)

0


In [65]:
infile = open("pickle_files\labels_list.pkl",'rb')
labels_id = pickle.load(infile)
infile.close()

In [66]:
id_labels = dict()
for class_name,idx in labels_id.items():
    id_labels[idx] = class_name
print(id_labels)
ypred_class = int(ypred_class)
res = id_labels[ypred_class]

{0: 'c8', 1: 'c0', 2: 'c6', 3: 'c7', 4: 'c9', 5: 'c1', 6: 'c3', 7: 'c5', 8: 'c4', 9: 'c2'}


In [67]:
class_name = dict()
class_name["c0"] = "SAFE_DRIVING"
class_name["c1"] = "TEXTING_RIGHT"
class_name["c2"] = "TALKING_PHONE_RIGHT"
class_name["c3"] = "TEXTING_LEFT"
class_name["c4"] = "TALKING_PHONE_LEFT"
class_name["c5"] = "OPERATING_RADIO"
class_name["c6"] = "DRINKING"
class_name["c7"] = "REACHING_BEHIND"
class_name["c8"] = "HAIR_AND_MAKEUP"
class_name["c9"] = "TALKING_TO_PASSENGER"

print(class_name[res])

HAIR_AND_MAKEUP


In [18]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

img = Image.open('car_test_2.jpg')
I1 = ImageDraw.Draw(img)
font = ImageFont.truetype("arial.ttf", 50)
I1.text((28, 36), class_name[res], font=font, fill=(0, 255, 0))
img.save("car_1.jpg")

In [75]:
from keras.models import load_model
vgg_model = load_model('model/vgg16/distracted-168-0.92.hdf5')

In [86]:
def path_to_tensor_vgg(img_path):
    # loads RGB image as PIL.Image.Image type
    #img = image.load_img(img_path)
    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    # convert PIL.Image.Image type to 3D tensor with shape (128, 128, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 128, 128, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [129]:
infile = open("pickle\labels_list_vgg16.pkl",'rb')
new_dict = pickle.load(infile)
infile.close()
print(new_dict)

{'c3': 0, 'c4': 1, 'c1': 2, 'c8': 3, 'c2': 4, 'c6': 5, 'c7': 6, 'c0': 7, 'c9': 8, 'c5': 9}


In [130]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 
vgg_test_tensor = path_to_tensor_vgg('car_test_8.jpg').astype('float32')/255 - 0.5

In [131]:
ypred_test = model.predict(vgg_test_tensor,verbose=1)
ypred_class = np.argmax(ypred_test,axis=1)

1/1 [==============================] - 0s 68ms/step


In [132]:
print(ypred_class)

[4]
